# Chapter 7
## Moving Beyond Linearity

Relaxing the linearity assumption while still maintaining as much interpretibility as possible.

1. __Polynomial Regression__ :- Extends the linear model by adding extra predictors, obtained by raising each of the original predictors to a power.


2. __Step Functions__ :- Cut the range of a variable into K distinct regions in order to produce a qualitative variable. This has the effect of fitting a piecewise constant function.


3. __Regression Splines__ :- Dividing a variable to K distinct regions. Within each region, a polynomial is fit to data. The polynomial join smoothly at region boundaries called knots. 


4. __Smoothing Splines__ :- They result from minimizing a RSS criterion subject to smoothness penalty.


5. __Local Regression__ :- Similar to splines but the region are allowed to overlap.


6. __Generalized Additive Models__ :- Extend the above method to deal with multiple predictors.

### Polynomial Regression 
A polynomial regression model can be written as 
$$y_i = \beta_0 + \beta_1 x_{i} +  \beta_2 x_{i}^{2} +....+ \beta_d x_{i}^{d} + \epsilon_i$$
$\epsilon_i$ = error term.

large value of d means more flexible curve especially near the boundary of X. hence it is unusual to use d greater than 3 or 4. 

The coefficients can be estimated using least squares. Least squares returns varaince estimates for each fitted coefficients $\hat \beta_j$

If $l_o$ is the point estimate = $(1,x_0,x_{0}^{2},..)$ and C is the covaraince matrix then pointwise standard error can be written as $l_{0}^{T}Cl_0$.
This can be used to estimate the CI.
